# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [1]:
from string import punctuation
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 

In [2]:
dvach = open('2ch_corpus.txt', encoding='utf-8').read()

In [3]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


In [4]:
norm_dvach = normalize(dvach)

In [5]:
from collections import Counter

In [6]:
vocab_dvach = Counter(norm_dvach)

In [7]:
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})

In [8]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [9]:
sents = sent_tokenize(dvach)

In [10]:
sentences_dvach = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sents]

In [244]:
sents_train, sents_test = sentences_dvach[:-30], sentences_dvach[-30:]

In [245]:
sents_train

[['<start>',
  '<start>',
  'анимублядский',
  'webm-треддля',
  'приличных',
  'анимублядей',
  'и',
  'прочих',
  'аутистов',
  '<end>'],
 ['<start>',
  '<start>',
  'безграмотное',
  'быдло',
  'с',
  'дубляжом',
  'войсовером',
  'порнографией',
  'и',
  'котиками',
  'советы',
  'мерзких',
  'мокрописечников',
  'вниманиебляди',
  'всех',
  'видов',
  'и',
  'прочее',
  'непотребство',
  'отправляется',
  'в',
  'порнотред',
  'ссылка',
  'для',
  'поиска',
  'с',
  'o',
  'уса',
  'видео',
  'сохраняем',
  'кадр',
  'правый',
  'клик',
  'по',
  'видео',
  'и',
  'ищем',
  'его',
  'на',
  'для',
  'воспроизведения',
  'webm',
  'с',
  '10-битным',
  'цветом',
  'нужно',
  'установить',
  'плагин',
  'vlc',
  'и',
  'отключить',
  'встроенный',
  'в',
  'браузер',
  'плеер',
  'media',
  '<end>'],
 ['<start>', '<start>', 'webm', '<end>'],
 ['<start>',
  '<start>',
  'enabled',
  'false',
  'в',
  'firefox',
  'о',
  'кодировании',
  'webm',
  'доступные',
  'кодеки',
  '—',
  'vp

In [12]:
import json
with open('data.json', 'w') as f:
    json.dump(sentences_dvach, f)

In [246]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sents_train:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))

In [119]:
from scipy.sparse import lil_matrix, csc_matrix, csr_matrix

In [247]:
matrix_dvach = lil_matrix((len(bigrams_dvach), 
                   len(unigrams_dvach)))
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {bigr:i for i, bigr in enumerate(id2bigram_dvach)}

for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    bigram = word1+' '+word2
    matrix_dvach[bigram2id_dvach[bigram],word2id_dvach[word3]] = (trigrams_dvach[ngram]/
                                                                        bigrams_dvach[bigram])
    #print(trigrams_dvach[ngram]/bigrams_dvach[bigram])



In [248]:
matrix_dvach = csr_matrix(matrix_dvach)

In [249]:

def generate(matrix, id2word, word2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id_dvach[start]
    chosen = start.split()[1]
    for i in range(n):
        last_chosen=chosen
        chosen = id2word[np.random.choice(list(range(matrix.shape[1])), p=matrix[current_idx].toarray()[0])]
        text.append(chosen)
        #print(text)
        #print(chosen)
        if chosen == '<end>':
            last_chosen, chosen = start.split()
        current_idx = bigram2id_dvach[last_chosen + ' ' + chosen]
    
    return ' '.join(text)

In [250]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach).replace('<end>', '\n'))

хуже этой хуйни сильно проигрывал раньше когда еще не одет в тренировочной одежде 
 что за конфликт 
 archiveinvalidation нужна версия с альбома ophidian a мне больше никто из быстрозаймов не хочет лишиться премии горничная уебалась в туалете испугались 
 что производительнее 
 вот когда не понимал как эта пизда шлюха шлюхой но считаю что бате не стоит внимания ну что еще 
 а то я не военный дебилы потомушто 
 ябать ты знаток криминальных профессий лол 
 во-первых функция не находится другой анон если будет чо непонятно кидай в тред и пойдут дальше ужас-то какой 
 твой первый дом


In [251]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach).replace('<end>', '\n'))

зарабатываешь дохера денег и виллы на берегу океана да мамка туда не завезли 
 два пакета анусов и твоя манера общения вдохновили меня на людскую сознательность ну а мы еще посидим и кароч я еще жить года полтора минимум 
 зачем машу испортил гад 
 какие инструмент подойдут для аутиста поясните 
 скопировал из вконтакта протекло 
 потом стилизуй как label before не падать 
 да хоть официально себя султаном провозгласит не слишком понравится украинскому президенту петру порошенко 
 как пройти первую миссию с са дают ненужный обрез а со своим внутренним миром 
 мотя кек попахивает гейством 
 член


In [252]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach).replace('<end>', '\n'))

я не считаю что нужно или нет вопрос другой 
 сижу пиздастрадаю 
 курящих убеждают не курить поднимая цены пьющих не пить не есть весомым для меня эти дни 
 а вот это двачую 
 дуриан ёбаный 
 русифицировал ваше говно пусть реализуют васяны в своих модах 
 шлюха успокойся никто не запрещает 
 cameltoe с трусами не бывает щас будет тройка значит можно порвать руками а прощалыкина изъяла у нее союз кто в картачки играть будет вообще ни с кем будет биться 
 хм интересно конечно 
 на самом деле есть один градиент как мне хуй сюда 
 бедненькая


In [253]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach).replace('<end>', '\n'))

и никто так и высказал одной свой подруге что у меня сохранилась какая-то отрицательная ассоциация со всем остальным что то пропустил 
 слепой блядь 10 487 443 байт считать умеешь 
 я ей 8 ставил 
 а ты где подрыв увидел пидорашнутый а из этого драгметалла 
 да снимай штанишки сладенький 
 не спорь придаток к жктбля сложнаэто че пиздец вы тугодумы 
 мы осмелели и вытащили свои стручки из трусов 
 а будет гонка братишка я тебе могу привести тебе аргумент если ты родился в нормальной стране не родилась 
 мечети при первом пиздеце будь то что можно добиться


In [254]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N) 

In [285]:
np.exp(np.sum(all_probas))

0.0

In [286]:
all_probas = []
for sent in sents_train:
    probs = []
    for ngram in ngrammer(sent, n=3):
        word1, word2, word3 = ngram.split()
        bigram = word1+' '+word2
        if bigram in bigrams_dvach and ngram in trigrams_dvach:
            probs.append(np.log(matrix_dvach[bigram2id_dvach[bigram],word2id_dvach[word3]]))
        else:
            probs.append(np.log(0.00001))
    all_probas.append(probs)

Перплексия

In [288]:
np.mean([perplexity(p) for p in all_probas])

22.128889435748945

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

В случае, если состав словаря определен до этапа обучения, можно все незнакомые слова заменить условным тегом <UNK> (неизвестный, ну или любым другим), и тогда (подобно тому, как иногда одиночные кластеры собирают в один большой), у этого тега при обучении появится своя собственная вероятность. 

Если же вокабуляр изначально не определен, можно на этапе обучения заменять <UKN> лексемы, не перешедшие некий порог частотности (либо, если размер словаря установлен, не попавшие в топ-V, где V - размер словаря), а потом соответственно считать их суммарную вероятность.

Так или иначе, у обоих этих методов есть слабое место - текст будет казаться еще более несогласованным. Более справедливым решением было бы заменить <UKN> на целое семейство неизвестных тегов с грамматическими фичами, но это может замедлить генерацию:)



2. Что такое сглаживание (smoothing)?

Поскольку тренировочные сеты не бесконечны, вполне возможно, что какие-то сочетания будут сверхчастотными, а какие-то (например, не встретившаяся ни разу биграмма, состоящая из имеющихся в корпусе униграмм) и вовсе не встретятся ни разу. Чтобы решить эту проблему "ограниченности" и не присваивать нулевую вероятность явлению, которое могло просто не встретиться в корпусе для обучения, и используется сглаживание.

Для этого можно прибавлять к вероятности (до деления на количество возможных n-грамм) единицу (Лаплас), несколько, можно использовать своего рода наследование (квадраграммы->триграммы->биграммы->униграммы)